In [ ]:
import re
import pandas as pd

In [ ]:
context_variables = {
    'partner':["wife","husband","girlfriend","boyfriend","spouse","honeymoon","husband","fiance"],
    'lunch':["lunch","noon"],
    'birthday':["birthday","bday","b'day"],
    'solo':["solo","alone"],
    'family':["father","mother","parents","son","daughter","child","children","offspring","brother","sister","sibling","twin","mom","dad","uncle","aunt","nephew","niece","cousin","family","relative"],
    "colleague":["colleague","boss","business","worker","conference"],
    "anniversary":["anniversary"],
    "promotion":["promotion","coupon","discount","offer"],
    "holiday":["holiday","vacation","trip","newyear","christmas","xmas","x'mas","thanksgiving"],
    "breakfast":["breakfast","morning"],
    "dinner":["dinner","night"],
    "brunch":["brunch"],
    "snacks":["snack","tea", "coffee"],
    "drinks":["drink","bar"]
}


In [ ]:
import json
with open('yelp_academic_dataset_review.json',encoding="utf8") as f:
    data = [json.loads(line) for line in f]


In [ ]:
data_subset=data[:10000]
len(data_subset)
#col_names[0]

In [ ]:
context_data_list = list()
for review in data_subset:
    rev_dict = {'review_id':review['review_id'],
                'business_id':review['business_id'],
                'stars':review['stars']}
    for var in context_variables:
        isPresent = 0
        for value in context_variables[var]:
            if re.search('(?<!\w){}(?!\w)'.format(value),review['text']):
                isPresent = 1
            rev_dict[var]=isPresent
    context_data_list.append(rev_dict.copy())

In [ ]:
context_data_df = pd.DataFrame(context_data_list)
context_data_df.shape

In [ ]:
context_data_df.describe()

In [ ]:
context_data_df.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
X_train,X_test=train_test_split(context_data_df,test_size=0.2)

In [ ]:
for n in list(X_train.index):
    X_train.loc[n,'Avg']=X_train[X_train['business_id'] == X_train.loc[n,'business_id']]['stars'].mean(axis=0)
for n in list(X_test.index):
    X_test.loc[n,'Avg']=X_test[X_test['business_id'] == X_test.loc[n,'business_id']]['stars'].mean(axis=0)

In [ ]:
X_train=X_train.drop(['business_id', 'review_id'], axis=1)
X_test=X_test.drop(['business_id', 'review_id'], axis=1)

In [ ]:
y_train=X_train.pop('Avg')
y_test=X_test.pop('Avg')

In [ ]:
regr = RandomForestRegressor(max_depth=16, random_state=32,n_estimators=1000)
regr.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
results = regr.predict(X_test)
rmse=sqrt(mean_squared_error(y_test, results))
print(rmse)
print(regr.feature_importances_)
print(regr.score(X,y))

In [ ]:
X1=my_df[['Rating']]
y1=my_df['Avg']

In [ ]:
X1_train,X1_test,y1_train,y1_test=train_test_split(X1,y1,test_size=0.2)

In [ ]:
regr1 = RandomForestRegressor(max_depth=16, random_state=32,n_estimators=1000)
regr1.fit(X1_train, y1_train)

In [ ]:
results1 = regr1.predict(X1_test)
rmse1=sqrt(mean_squared_error(y1_test, results1))
print(rmse1)
#print(regr1.feature_importances_)
print(regr1.score(X1,y1))

In [ ]:
drmse=rmse1-rmse
print(drmse)